###           Classification of Facial Expression using Feed Forward Network(Perceptron)

#### Data Preprocessing Part

In [1]:
#Importing Libraries
import os
import glob
import pickle
import random
import numpy as np
import cv2
import matplotlib.pyplot as plt


Bad key "text.kerning_factor" on line 4 in
C:\Users\jayan\Anaconda\envs\tf\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
#Getting the files
folders = [name for name in glob.glob('Dataset/*') if os.path.isdir(name)]
folders

['Dataset\\fortuner', 'Dataset\\swift', 'Dataset\\vento']

In [3]:
classes = []
for directory in folders:
    classes.append(directory.split('\\')[1])
classes

['fortuner', 'swift', 'vento']

In [4]:
#Nested tuples,dictionaries,lists are the form of dataset
training_data = []                      #The data used for training purpose
for directory in folders:
    paths = glob.glob(directory+'/*.jpg')
    current_class = directory.split('\\')[1]
    for path in paths:
        training_dictionary = {}       
        training_dictionary['path'] = path
        training_data.append((current_class,training_dictionary))

In [5]:
training_data

[('fortuner',
  {'path': 'Dataset\\fortuner\\1041e9c9ad54fea972391e21133d09eb_large.jpg'}),
 ('fortuner',
  {'path': 'Dataset\\fortuner\\113946013-chiangmai-thailand-november-8-2018-private-toyota-fortuner-suv-car-on-road-no-1001-8-km-from-chiangm.jpg'}),
 ('fortuner',
  {'path': 'Dataset\\fortuner\\12dfcf72d2d3b45d85e1dfa9f6e5cab8_extra_large.jpg'}),
 ('fortuner',
  {'path': 'Dataset\\fortuner\\1550225910_Toyota_Fortuner_in_Pattaya.JPG.jpg'}),
 ('fortuner', {'path': 'Dataset\\fortuner\\1902990_9593_1_1583399905356.jpg'}),
 ('fortuner', {'path': 'Dataset\\fortuner\\1914992_13981_1587215363172.jpg'}),
 ('fortuner',
  {'path': 'Dataset\\fortuner\\2015_Toyota_Fortuner_28New_Zealand29.jpg'}),
 ('fortuner',
  {'path': 'Dataset\\fortuner\\2018-Toyota-Fortuner-TRD-Featured.jpg'}),
 ('fortuner',
  {'path': 'Dataset\\fortuner\\2020-toyota-fortuner-bs6-1-1068x645.jpg'}),
 ('fortuner',
  {'path': 'Dataset\\fortuner\\267844bb085fcee1130a162ec9dc47d2.jpg'}),
 ('fortuner',
  {'path': 'Dataset\\fortu

In [6]:
random.shuffle(training_data)

In [7]:
with open('training_data.pkl','wb') as f:
    pickle.dump(training_data,f)
with open('classes.pkl','wb') as f:
    pickle.dump(classes,f)

#### Designing the Network

In [8]:
#Retrievng the Stored Data
with open('training_data.pkl','rb') as f:
    training_data = pickle.load(f)
with open('classes.pkl','rb') as f:
    classes = pickle.load(f)

### Training Algorithm Begins

In [34]:
# Constants
bias = 1
Train_Test_Ratio = 0.6
Iterations = 10
lr = 0.0834

In [35]:
def sigmoid(x):
    return (1/1+np.exp(-x))

def ReLu(x):
    return max(0,x)       

In [36]:
#Defining the Perceptron Class

class MultiClassPerceptron:
    accuracy = 0

    def __init__(self,classes,training_data,train_test_ratio=Train_Test_Ratio,iterations=Iterations):
        self.classes = classes
        self.training_data = training_data
        self.ratio = train_test_ratio
        self.iterations = iterations

        #Splitting Feature Data into Training and Testing Set For The Second step i;e training pair s:t
        random.shuffle(self.training_data)
        self.train_set = self.training_data[:int(len(self.training_data)*self.ratio)]
        self.test_set = self.training_data[int(len(self.training_data)*self.ratio):]

        #Initialising the Weight Vectors in the Form of a Dictionary with BIAS terms
        self.weight_vectors = {c:np.array([0.0 for _ in range(9217)]) for c in self.classes}

    def train(self):

        for i in range(self.iterations):
            for category,training_dictionary in self.train_set:
                try:
                    image = cv2.imread(training_dictionary['path'])  #Obtaininig the Image and preprocessing it
                    #print(training_dictionary['path'])
                    
                    
                    dim = (96,96)     #Dimension Required for Resizing the Data
                    new_image = cv2.resize(image,dim,cv2.INTER_AREA)
                    
                    input_array = []
                    input_array = [np.mean(new_image[i][j]/255) for i in range(new_image.shape[0])
                                  for j in range(new_image.shape[1])]
                    input_array.append(bias)
                    input_vector = np.array(input_array)

                    arg_max,predicted_class = 0,self.classes[0]

                    for index in self.classes:
                        current_activation = np.dot(input_vector,self.weight_vectors[index])
                        #current_activation = ReLu(current_activation)
                        if current_activation >= arg_max:
                            arg_max,predicted_class = current_activation,index

                    if not(category == predicted_class):
                        self.weight_vectors[category] += [weight*lr for weight in input_vector]
                        self.weight_vectors[predicted_class] -= [weight*lr for weight in input_vector]
                except :
                    print("E")
            print(i,'th iteraion')
            self.calculate_accuracy()

    def predict(self,training_dictionary):
        img = cv2.imread(training_dictionary['path'])
        dim = (96,96)
        new_image = cv2.resize(img,dim,cv2.INTER_AREA)
        
        input_array = []

        input_array = [np.mean(new_image[i][j]/255) for i in range(new_image.shape[0])
                      for j in range(new_image.shape[1])]

        input_array.append(bias)
        feature_vector = np.array(input_array)
        arg_max,predicted_class = 0,self.classes[0]

        for index in self.classes:
            current_activation = np.dot(feature_vector,self.weight_vectors[index])
            if current_activation >= arg_max:
                arg_max,predicted_class = current_activation,index
        return predicted_class
    
    def predict_img(self,path):
        img = cv2.imread(path)
        dim = (96,96)
        new_image = cv2.resize(img,dim,cv2.INTER_AREA)
        
        input_array = []

        input_array = [np.mean(new_image[i][j]/255) for i in range(new_image.shape[0])
                      for j in range(new_image.shape[1])]

        input_array.append(bias)
        feature_vector = np.array(input_array)
        arg_max,predicted_class = 0,self.classes[0]

        for index in self.classes:
            current_activation = np.dot(feature_vector,self.weight_vectors[index])
            if current_activation >= arg_max:
                arg_max,predicted_class = current_activation,index
        return predicted_class

    def test_random_data(self):
        item = random.choice(self.test_set)
        print("Actual Class :" + item[0])
        pred_class = self.predict(item[1])
        print("Predicted Class :" + pred_class)

    def calculate_accuracy(self):
        correct,incorrect = 0,0
        random.shuffle(self.training_data)
        self.test_set = self.training_data[int(len(self.training_data)*self.ratio):]
        for training_dictionary in self.test_set:
            actual_class = training_dictionary[0]
            predicted_class = self.predict(training_dictionary[1])
            if actual_class == predicted_class:
                correct += 1
            else:
                incorrect += 1
        print("ACCURACY")
        print("Accuracy of FF Model is ",(correct * 1.0) / ((correct + incorrect) * 1.0))

    def select_random_picture(self):
        s = glob.glob(os.getcwd() + '\\Testing_Sample/*')

        img = cv2.imread(s[0])
        dim = (96,96)
        new_image = cv2.resize(img,dim,cv2.INTER_AREA)

        input_array = []

        input_array = [np.mean(new_image[i][j]/255) for i in range(new_image.shape[0])
                      for j in range(new_image.shape[1])]
        input_array.append(bias)
        feature_vector = np.array(input_array)

        arg_max,predicted_class = 0,self.classes[0]
        for index in self.classes:
            current_activation = np.dot(feature_vector,self.weight_vectors[index])
            if current_activation >= arg_max:
                arg_max,predicted_class = curren[t_activation,index]

        print("The Predicted Class for the Sample Image is")
        print(predicted_class)

In [37]:
if __name__ == "__main__":
    classifier = MultiClassPerceptron(classes,training_data)
    classifier.train()
    classifier.calculate_accuracy()

0 th iteraion
ACCURACY
Accuracy of FF Model is  0.24528301886792453
1 th iteraion
ACCURACY
Accuracy of FF Model is  0.29559748427672955
2 th iteraion
ACCURACY
Accuracy of FF Model is  0.29559748427672955
3 th iteraion
ACCURACY
Accuracy of FF Model is  0.33962264150943394
4 th iteraion
ACCURACY
Accuracy of FF Model is  0.2641509433962264
5 th iteraion
ACCURACY
Accuracy of FF Model is  0.4968553459119497
6 th iteraion
ACCURACY
Accuracy of FF Model is  0.559748427672956
7 th iteraion
ACCURACY
Accuracy of FF Model is  0.5283018867924528
8 th iteraion
ACCURACY
Accuracy of FF Model is  0.2893081761006289
9 th iteraion
ACCURACY
Accuracy of FF Model is  0.5094339622641509
ACCURACY
Accuracy of FF Model is  0.5220125786163522


In [60]:
classifier.test_random_data()

Actual Class :vento
Predicted Class :swift


In [54]:
classifier.predict({'path': 'Dataset\\vento\\images75.jpg'})

'swift'

In [47]:
classifier.test_set[0][1]

{'path': 'Dataset\\vento\\images72.jpg'}

In [55]:
classifier.predict_img('Dataset\\vento\\images75.jpg')

'swift'